In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import required Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import seaborn as sb

## Read Data

In [ ]:
input_dir = '/kaggle/input'
isic_dir = os.path.join(input_dir, 'siim-isic-melanoma-classification')
jpg_dir = os.path.join(isic_dir, 'jpeg')
work_dir = os.path.abspath(os.getcwd())

In [ ]:
train_csv = pd.read_csv(os.path.join(isic_dir, 'train.csv'))
test_csv = pd.read_csv(os.path.join(isic_dir, 'test.csv'))

train_csv.head()

In [ ]:
test_csv.head()

In [ ]:
def preprocess(data):
    data['sex'] = data['sex'].astype('category')
    data['site'] = data['anatom_site_general_challenge'].astype('category')
    data['diagnosis'] = data['diagnosis'].astype('category')
    data['benign_malignant'] = data['benign_malignant'].astype('category')
    
    return data

In [ ]:
train_csv = preprocess(train_csv)

# Explore Data

## Check Missing Values

In [ ]:
train_csv.info()

In [ ]:
total = train_csv.shape[0]
print('Missing sex info: ', total - train_csv[train_csv.sex.notnull()].shape[0])
print('Missing age info: ', total - train_csv[train_csv.age_approx.notnull()].shape[0])
print('Missing site info: ', total - train_csv[train_csv.site.notnull()].shape[0])

### Check if missing has any relationship with malign/benign

In [ ]:
train_csv['has_missing'] = train_csv.apply(lambda row: row.isna().any(), axis=1)

In [ ]:
train_csv.head()

In [ ]:
sb.distplot(train_csv[train_csv.has_missing==True]['target'], label='Missing', kde=False)
sb.distplot(train_csv[train_csv.has_missing==False]['target'], label='Full', kde=False)
plt.legend(loc='best')
plt.show()

In [ ]:
missing_malignant = train_csv[(train_csv.has_missing==True) & (train_csv.target==1)].shape[0]
missing_benign = train_csv[(train_csv.has_missing==False) & (train_csv.target==1)].shape[0]

print('Malignant with missing values: ', missing_malignant)
print('Benign with missing values: ', missing_benign)

## Check for duplicates

In [ ]:
unique_patients = train_csv['patient_id'].unique().shape[0] 
unique_images = train_csv['image_name'].unique().shape[0]
total = train_csv['patient_id'].shape[0]

print('Unique patients: ', unique_patients)
print('Unique images: ', unique_images)
print('Total: ', total)

In [ ]:
# for train and test
common_images = test_csv['image_name'].isin(train_csv['image_name']).value_counts()
common_patients = test_csv['patient_id'].isin(train_csv['patient_id']).value_counts()
print('Common images ---> ', common_images)
print('Common Patients ---> ', common_patients)
print('Total test: ', test_csv.shape[0])

Looks like 2056 patients have 33126 images. Let's see the distribution of images per person. Count of patient ID should do the job.

In [ ]:
sb.distplot(train_csv["patient_id"].value_counts())
plt.title("Patient ID distribution")

### Histograms for each variables

In [ ]:
# For numeric columns
plt.rcParams['figure.figsize'] = 10,4
train_csv['age_approx'].hist()
plt.show()

In [ ]:
# For non-numeric columns
plt.rcParams['figure.figsize'] = 16,12
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
train_csv['sex'].value_counts().plot(kind='bar', ax=ax1, title='Sex')
train_csv['anatom_site_general_challenge'].value_counts().plot(kind='bar', ax=ax2, title='Site')
train_csv['diagnosis'].value_counts().plot(kind='bar', ax=ax3, title='Diagnosis')
train_csv['benign_malignant'].value_counts().plot(kind='bar', ax=ax4, title='Benign/Malignant')
plt.tight_layout() 
plt.show()

### Finding from histogram
We can see that the dataset is highly imbalanced with very few malignant cases. We have to be careful while selecting optimization metric. Using accuracy might give us misleading result. ROC curve and PR curve will do a better job for selecting optimum model.

### Plot Across features

In [ ]:
# plt.rcParams['figure.figsize'] = 6,4

plt.rcParams['figure.figsize'] = 16,6
fig, (ax1, ax2) = plt.subplots(1,2)
sb.countplot(x='sex', hue='benign_malignant', data=train_csv, ax=ax1)
sb.countplot(x='anatom_site_general_challenge', hue='benign_malignant', data=train_csv, ax=ax2)
plt.tight_layout()
plt.show()

In [ ]:
sex_site_result = train_csv.groupby(['sex', 'site', 'benign_malignant']).agg(occurence=('target','count'))
sex_site_result = sex_site_result.reset_index()
sex_site_result

In [ ]:
sex_site_result.info()

In [ ]:
sb.scatterplot(x='site', y='occurence', hue='benign_malignant', style='sex', data=sex_site_result)
plt.show()

In [ ]:
sb.scatterplot(x='age_approx', y='site', hue='benign_malignant', style='sex', alpha=0.6, markers=['P', 'X'], data=train_csv)
plt.show()

Looks like females above 80 and males above 60 are more vulnerable and it is less likely to appear in the oral/genital area. Let's have a look at age.

In [ ]:
# sb.pairplot(train_csv[['age_approx','benign_malignant']], hue="benign_malignant")
sb.distplot(train_csv[train_csv.target==0]['age_approx'], label='Benign')
sb.distplot(train_csv[train_csv.target==1]['age_approx'], label='Malignant')
plt.legend(loc='best')
plt.show()

In [ ]:
sb.distplot(train_csv[train_csv.sex=='male']['age_approx'], label='Male')
sb.distplot(train_csv[train_csv.sex=='female']['age_approx'], label='Female')
plt.legend(loc='best')
plt.show()

In [ ]:
sb.distplot(train_csv[(train_csv.sex=='male') & (train_csv.target==0)]['age_approx'], label='Male Benign')
sb.distplot(train_csv[(train_csv.sex=='male') & (train_csv.target==1)]['age_approx'], label='Male Malignant')
sb.distplot(train_csv[(train_csv.sex=='female') & (train_csv.target==0)]['age_approx'], label='Female Benign')
sb.distplot(train_csv[(train_csv.sex=='female') & (train_csv.target==1)]['age_approx'], label='Female Malignant')
plt.legend(loc='best')
plt.show()

From these plots we can conclude that:
1. Male and Female Benign distribution is similar
2. Malignant type is mostly prevalent in higher age group
3. Males over 60 are particularly more vulnerable.

## Compare images

In [ ]:
benign = train_csv[train_csv.benign_malignant=='benign']
malignant = train_csv[train_csv.benign_malignant=='malignant']

In [ ]:
bimg = plt.imread(jpg_dir + '/train/' + benign.iloc[0]['image_name'] + '.jpg')
plt.imshow(bimg)

In [ ]:
mimg = plt.imread(jpg_dir + '/train/' + malignant.iloc[0]['image_name'] + '.jpg')
plt.imshow(mimg)

In [ ]:
def show_rgb_histogram(image, ax=None):
    if ax is None:
        plt.hist(image[:, :, 0].ravel(), bins = 256, color = 'Red', alpha = 0.5, label='Red')
        plt.hist(image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5, label='Green')
        plt.hist(image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5, label='Blue')
        plt.xlabel('Intensity')
        plt.ylabel('Count')
        plt.legend(loc='best')
        plt.show()
    else:
        ax.hist(image[:, :, 0].ravel(), bins = 256, color = 'Red', alpha = 0.5, label='Red')
        ax.hist(image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5, label='Green')
        ax.hist(image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5, label='Blue')
        ax.set_xlabel('Intensity')
        ax.set_ylabel('Count')
        ax.legend(loc='best')

In [ ]:
show_rgb_histogram(bimg)

In [ ]:
show_rgb_histogram(mimg)

### Randomly pick some images and show their histogram

In [ ]:
random_benign = benign.sample(6)
random_malignant = malignant.sample(6)

In [ ]:
plt.rcParams['figure.figsize'] = 10,6
for index, row in random_benign.iterrows():
    img = plt.imread(jpg_dir + '/train/' + row['image_name'] + '.jpg')
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(img)
    show_rgb_histogram(img, ax2)
    plt.tight_layout()
    plt.show()

In [ ]:
for index, row in random_malignant.iterrows():
    img = plt.imread(jpg_dir + '/train/' + row['image_name'] + '.jpg')
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(img)
    show_rgb_histogram(img, ax2)
    plt.tight_layout()
    plt.show()

From the sample it looks like histogram of malignant looks a bit more spread out, but we cannot make that conclusion with just few sample. Anyway, it gives us a rough idea about color distribution which might be helpful if we need a specific filter for specific color.